### **Cargue de datos**


A continuación se importan los modulos que se usaran a lo largo del desarrollo del entendimiento de los datos.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.types import StructType
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import FloatType, StringType, IntegerType, DateType
from pyspark.sql.functions import udf, col, length, isnan, when, count
from pyspark.sql.functions import col, to_date
import pyspark.sql.functions as f
from pyspark.sql.types import NumericType
import os 
from datetime import datetime
from pyspark.sql import types as t
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
from pyspark.sql.functions import col

Tambien se definen las credenciales de acceso a la base de datos.

In [2]:
# Given Credentials
db_user = 'Estudiante_65_202415'
db_psswd = 'Estudiante_202010409'

connection_properties = {
    "user": db_user,
    "password": db_psswd,
    "driver": "com.mysql.cj.jdbc.Driver"
}

default_string_connection = 'jdbc:mysql://157.253.236.120:8080/WWImportersTransactional'

In [3]:
class MySQLConnector:
    def __init__(self, spark: SparkSession, connection_properties: dict, url: str):
        self.spark = spark
        self.properties = connection_properties
        self.url = url

    def get_dataframe(self, sql_query: str):
        """
        Execute a SQL query and return the result as a Spark DataFrame.

        Parameters:
        sql_query (str): SQL query to be executed.

        Returns:
        DataFrame: Resultant DataFrame from the executed SQL query.
        """
        df = self.spark.read.jdbc(
            url=self.url,
            table=sql_query,
            properties=self.properties
        )
        return df
    
def plot_numeric_box_plots(spark_df, sample_fraction=0.1):
    """
    Genera diagramas de caja para todas las columnas numéricas de un DataFrame de Spark.

    Parámetros:
    - spark_df: DataFrame de Spark.
    - sample_fraction: Fracción de datos a muestrear (valor entre 0 y 1). Por defecto es 1.0 (todos los datos).
    Retorna:
    - None. La función muestra los diagramas de caja.
    """
    # Identificar las columnas numéricas
    numeric_columns = [field.name for field in spark_df.schema.fields if isinstance(field.dataType, NumericType)]

    if not numeric_columns:
        print("El DataFrame no contiene columnas numéricas.")
        return

    if sample_fraction < 1.0:
        spark_df = spark_df.sample(fraction=sample_fraction)

    spark_numeric_df = spark_df.select(*numeric_columns)

    pandas_df = spark_numeric_df.toPandas()
    pandas_df = pandas_df.dropna()

    sns.set(style="whitegrid")

    num_columns = len(numeric_columns)
    fig, axes = plt.subplots(nrows=1, ncols=num_columns, figsize=(5 * num_columns, 6))

    if num_columns == 1:
        axes = [axes]

    for ax, column in zip(axes, numeric_columns):
        sns.boxplot(y=pandas_df[column], ax=ax)
        ax.set_title(f'Diagrama de caja de {column}')
        ax.set_ylabel(column)
        ax.set_xlabel('')

    plt.tight_layout()
    plt.show()

def plot_correlation_heatmap(spark_df, sample_fraction=1.0):
    """
    Genera un mapa de calor de correlaciones para todas las columnas numéricas de un DataFrame de Spark.

    Parámetros:
    - spark_df: DataFrame de Spark.
    - sample_fraction: Fracción de datos a muestrear (valor entre 0 y 1). Por defecto es 1.0 (todos los datos).

    Retorna:
    - None. La función muestra el mapa de calor de correlaciones.
    """
    numeric_columns = [field.name for field in spark_df.schema.fields if isinstance(field.dataType, NumericType)]

    if not numeric_columns:
        print("El DataFrame no contiene columnas numéricas.")
        return

    if sample_fraction < 1.0:
        spark_df = spark_df.sample(fraction=sample_fraction)

    spark_numeric_df = spark_df.select(*numeric_columns)

    pandas_df = spark_numeric_df.toPandas()

    pandas_df = pandas_df.dropna()

    corr_matrix = pandas_df.corr()

    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
    plt.title('Mapa de calor de correlaciones')
    plt.show()


def plot_scatter(df, x_col, y_col, sample_fraction):
    """
    Genera un gráfico de dispersión entre dos columnas de un DataFrame de Spark.

    Parámetros:
    df (DataFrame): DataFrame de Spark con los datos.
    x_col (str): Nombre de la columna para el eje x.
    y_col (str): Nombre de la columna para el eje y.
    sample_fraction (float): Porcentaje de datos a muestrear (entre 0 y 1).
    """

    if sample_fraction <= 0 or sample_fraction > 1:
        raise ValueError("El parámetro sample_fraction debe estar entre 0 y 1.")

    sample_df = df.select(x_col, y_col).sample(fraction=sample_fraction)

    pandas_df = sample_df.toPandas()

    if pandas_df.empty:
        print("No hay datos para visualizar después del muestreo. Por favor, incrementa sample_fraction.")
        return

    if pandas_df[x_col].dtype == 'object':
        try:
            pandas_df[x_col] = pd.to_datetime(pandas_df[x_col])
        except Exception as e:
            print(f"Error al convertir {x_col} a datetime: {e}")
            return

    plt.figure(figsize=(12, 6))
    sns.scatterplot(data=pandas_df, x=x_col, y=y_col)
    plt.title(f"Gráfico de dispersión de {y_col} vs {x_col}")
    plt.xlabel(x_col)
    plt.ylabel(y_col)
    plt.xticks(rotation=45)  # Rotar etiquetas del eje x si es necesario
    plt.tight_layout()
    plt.show()

def create_spark_session(path_jar_driver):
    # Configuración de la sesión de Spark
    conf = SparkConf() \
            .set('spark.driver.extraClassPath', path_jar_driver)

    spark_context = SparkContext(conf=conf)
    sql_context = SQLContext(spark_context)
    spark = sql_context.sparkSession

    return spark

In [13]:
# Import reusable modules
#import modules.spark_session_module 
#import modules.mysql_connector
#import modules.mysql_connector

In [4]:
# WINDOWS - Compatible with VM  
path_jar_driver = 'C:\Program Files (x86)\MySQL\Connector J 8.0\mysql-connector-java-8.0.28.jar'
#path_jar_driver = 'D:\Librerias\mysql-connector-java-8.0.28.jar'

# LINUX users: to download the jar wget https://dev.mysql.com/get/Downloads/Connector-J/mysql-connector-java-8.0.28.tar.gz
#path_jar_driver = '/opt/mysql/lib/mysql-connector-java-8.0.28.jar'

In [5]:
#spark = modules.spark_session_module.create_spark_session(path_jar_driver)
spark = create_spark_session(path_jar_driver)

C:\Users\estudiante\anaconda3\envs\Tutoriales\lib\site-packages\pyspark\sql\context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [7]:
conn = MySQLConnector(spark=spark, connection_properties=connection_properties, url=default_string_connection)

In [8]:
sql_benefits = 'RaSaTransaccional.FuenteTiposBeneficio_Copia_E'

In [13]:
beneficios  = conn.get_dataframe(sql_benefits)

+-----------------+--------------------+--------------------+-----+---------------------+-----------------------+----------------------------------------+---------------------------------------+-----+
|IdTipoBeneficio_T|              Nombre|     UnidadDelLimite|EsEHB|EstaCubiertaPorSeguro|TieneLimiteCuantitativo|ExcluidoDelDesembolsoMaximoDentroDeLaRed|ExcluidoDelDesembolsoMaximoFueraDeLaRed|Fecha|
+-----------------+--------------------+--------------------+-----+---------------------+-----------------------+----------------------------------------+---------------------------------------+-----+
|              565|Nutritional Couns...|                    |   No|                  Yes|                     No|                                      No|                                    Yes| 2017|
|              795|Rehabilitative Sp...|Days per Benefit ...|  Yes|                  Yes|                    Yes|                                      No|                                    Yes| 2

## **Perfilamiento de datos**

### **Entendimiento de los datos**

A continuacion se realiza el entendimiento de los datos.

In [12]:
beneficios

DataFrame[IdTipoBeneficio_T: int, Nombre: string, UnidadDelLimite: string, EsEHB: string, EstaCubiertaPorSeguro: string, TieneLimiteCuantitativo: string, ExcluidoDelDesembolsoMaximoDentroDeLaRed: string, ExcluidoDelDesembolsoMaximoFueraDeLaRed: string, Fecha: int]

In [14]:
beneficios.show()

+-----------------+--------------------+--------------------+-----+---------------------+-----------------------+----------------------------------------+---------------------------------------+-----+
|IdTipoBeneficio_T|              Nombre|     UnidadDelLimite|EsEHB|EstaCubiertaPorSeguro|TieneLimiteCuantitativo|ExcluidoDelDesembolsoMaximoDentroDeLaRed|ExcluidoDelDesembolsoMaximoFueraDeLaRed|Fecha|
+-----------------+--------------------+--------------------+-----+---------------------+-----------------------+----------------------------------------+---------------------------------------+-----+
|              565|Nutritional Couns...|                    |   No|                  Yes|                     No|                                      No|                                    Yes| 2017|
|              795|Rehabilitative Sp...|Days per Benefit ...|  Yes|                  Yes|                    Yes|                                      No|                                    Yes| 2

Como puede observarse en el anterior dataframe, la tabla de beneficios contiene la siguiente información:
* `IdTipoBeneficio_T` que corresponde al identificador del identificador del tipo de beneficio.
* `Nombre` que corresponde a la descripción del tipo de beneficio.
* `UnidadDelLimite` xxxx.
* `EsEHB` xxxxxx.
* `EstaCubiertaPorSeguro` Esta columna indica si el tipo de beneficio esta cubierto por el seguro.
* `TieneLimiteCuantitativo` Esta columna señala si el tipo de beneficio tiene limite cuantitativo, toma valores Yes/No.
* `ExcluidoDelDesembolsoMaximoDentroDeLaRed` Esta columna indica si el tipo de beneficio esta excluido del desembolso maximo dentro de la red, toma valores Yes/No.
* `ExcluidoDelDesembolsoMaximoFueraDeLaRed` Esta columna indica si el tipo de beneficio esta excluido del desembolso maximo fuera de la red, toma valores Yes/No.
* `Fecha` Indica el año de vigencia del tipo de beneficio.

La primera columna corresponderia a la llave primaria de la tabla.

### **Revisión de reglas de negocio:** 

A continuación se muestran las reglas de negocio dadas que tienen relación con la tabla tipo de beneficios:

* Los tipos de beneficios con límite cuantitativo deben tener una cantidad límite diferente de cero en los planes que los ofrecen.
* Las fuentes FuenteAreasDeServicio_Copia_E y FuenteTiposBeneficio_Copia_E comparten información de los años 2017 al 2019
* La empresa comparte 5409 áreas de servicios y 170 tipos de beneficios.

### **Análisis de calidad de datos**

#### **Completitud y Validez de los datos**

#### **Cardinalidad**

#### **Consistencia de los datos**

### **Analisis Descriptivo**

### Conclusiones

Las conclusiones se van consolidad en un solo documento word.
